In [ ]:
# pipeline-api
import tempfile
from unstructured.documents.pdf import PDFPage, PDFDocument

In [ ]:
import os

DIRECTORY = os.getcwd()
SAMPLE_DOCS_DIRECTORY = os.path.join(DIRECTORY, "sample-docs")

filename = os.path.join(SAMPLE_DOCS_DIRECTORY, "fake-oer.pdf")

In [ ]:
# pipeline-api
import warnings

def partition_oer(filename: str):
    doc = PDFDocument(filename)

    # NOTE(robinson) - The warning we catch comes from the detectron2
    # code. We have an issue to contribute back a fix for that
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        doc._read()
        pages = [page for page in doc.pages]
        
    return pages

In [ ]:
pages = partition_oer(filename)

## Cleaning Brick

In [ ]:
for element in pages[0].elements:
    print(f"\n\n{element}")

In [ ]:
# pipeline-api
import re


BLOCK_TITLE_RE = re.compile(r"c. (SIGNIFICANT DUTIES AND RESPONSIBILITIES"
                            r"|COMMENTS ON POTENTIAL)")

def clean_block_titles(narrative: str) -> str:
    """Cleans the name of the block from the extracted narrative text"""
    return BLOCK_TITLE_RE.sub("", narrative).strip()

In [ ]:
clean_block_titles(pages[0].elements[0].text)

In [ ]:
for element in pages[1].elements:
    print(f"\n\n{element}")

### Staging

In [ ]:
# pipeline-api
COMMENT_BLOCKS = [
    "character",
    "presence",
    "intellect",
    "leads",
    "develops",
    "achieves",
]

In [ ]:
# pipeline-api
def structure_oer(pages):
    """Creates a dictionary with the extracted elements of the OER"""
    if len(pages) < 2:
        raise ValueError(f"Pages length is {len(pages)}). "
                          "Expected 2 pages.")
    
    structured_oer = dict()
        
    first_page = pages[0].elements 
    if len(first_page) < 2:
        raise ValueError(f"Number of narrative text elements on the "
                         f"first page is {len(first_page)}. "
                          "Expected at least two.")
    
    duty_description = clean_block_titles(first_page[0].text)
    structured_oer["duty_description"] = duty_description
    structured_oer["rater_comments"] = first_page[-1].text
    
    second_page = pages[1].elements
    num_sections = len(COMMENT_BLOCKS)

    if len(first_page) < 2:
        raise ValueError(f"Number of narrative text elements on the "
                         f"second page is {len(second_page)}. "
                         f"Expected at least {num_sections}.")
    
    for i, section in enumerate(second_page[:num_sections]):
        key = COMMENT_BLOCKS[i]
        structured_oer[key] = section.text
        
    structured_oer["intermediate_rater"] = second_page[-2].text
        
    return structured_oer

In [ ]:
oer = structure_oer(pages)

### API Definition

In [ ]:
# pipeline-api
def pipeline_api(text):
    tmp = tempfile.NamedTemporaryFile(prefix="tmp_", delete=False)
    tmp.write(text)
    tmp.close()
    
    pages = partition_oer(tmp.name)
    return structure_oer(pages)

In [ ]:
with open(filename, "rb") as f:
    oer = pipeline_api(f.read())

In [ ]:
import json

print(json.dumps(oer, indent=4))